# RA Yearly Stats

Welcome to RA Yearly Stats! This Jupyter notebook is a window to your yearly RetroAchievements stats. Do you want to know how many games you beat this last year? How many achievements you got? Maybe how they are distributed among the consoles or which dev gave you the most achievements? You're in for a treat!

This is a project I started just for the fun of learning how to make API calls that ended up in what you see now. At some point I was happy enough with the shape it was taking to decide that it might be of interest to people other than me. Ever since I discovered RetroAchievements, I wanted to repay the community in some way for all of the free fun that it has given me, so take this as my little thank you to you all.

## Instructions

In order to be able to show you this data, **RA Yearly Stats just requires your username and your API key**, which you can input in the code cell below this text block. You can find your API key in the RetroAchievements website, under Settings.

In that code cell, you will also find a variable called hardcore_mode_only set to False. You can replace this value with True (without any quotation marks) if you don't want to take achievements scored in Softcore Mode into account, which can speed up the process a bit.

Please input the username and the API key inside the provided quotation marks for the code to work properly. Then, hit the 'Run All Cells' button that is under 'Run' in the toolbar above and wait a bit for the program to make all the required requests to the RetroAchievements API and work on the data.

As a matter of fact, you can use your API key to not only check your data but also any other users', so feel free to check how other users are doing using your own API key!

## Considerations

- **The program requires you to have earned at least one achievement**, it will break otherwise. If you chose hardcore_mode_only to be True, then you are required to have earned at least one Hardcore Mode achievement.

- **You will be able to select any year you have RetroAchievements data for** using the selector located at the beginning of your showcase. The program will default to the last full year which, at the time of uploading the tool, is 2024. If there is no data for that year, it will default to the most recent year you have data of.

- **The program will take some time to load** because it waits a bit between calls to the API to avoid saturation. Please be patient, it usually takes less than a minute but it can take a few minutes for users with lots of achievements.

- If you **hover your mouse over the data of any graph**, it will show further detail.

- **The program will not show all elements properly in Jupyter's Dark Mode**. As a programmer, I swear a vow to fix this in the future, but I had to share the tool at some point or it would be stuck in my computer forever, there's always another feature required for perfection.

- If you are running this program on Binder, **it might be the case that you have too many achievements for the resources that Binder allocates** per instance. Please consider turning on the hardcore_mode_only flag or running the code locally in your computer if that's the case.

- This program was originally designed to work only with Hardcore Mode achievements since that is the mode I play on. However, I ended up adding Softcore Mode achievement support to avoid gatekeeping a good chunk of the community. Please note that there is no distinction between 'Mastered' (Hardcore) and 'Completed' (Softcore), nor between 'Beaten' and 'Beaten in Softcore Mode'. Also, in the daily point distribution graph, there is no distinction between points scored in Softcore Mode and Hardcore Mode.

- Please note that if an achievement that was earned in Softcore Mode is later earned in Hardcore Mode, its data is overwritten on the RetroAchievements side, and so it will appear as if it was never earned in Softcore Mode. This is something I can't work around so please do not think of it as a bug.

- I'm not a web developer but I tried to use some HTML code to have a better looking display of the information. However, I don't know how the HTML code will behave with any setup different than the one I use. For the record, **the code has been tweaked to work best on Google Chrome in 1920x1080 resolution. Please consider to use a similar setup for best results.**

- I've tried to test the program as thoroughly as possible, but issues are bound to happen once more people try to use the tool. **If you find a bug or have a suggestion, please contact me** either through RetroAchievements direct messages or by opening a GitHub issue and explain your issue in as much detail as possible so that I can work on it.

In [ ]:
### MODIFY THE CODE BELOW TO INPUT YOUR DATA ###

username = ""
api_key = ""
hardcore_mode_only = False

## DO NOT MODIFY ANY CODE BEYOND THIS POINT ###

In [ ]:
# Here's all the code hidden

import requests

import numpy as np
import pandas as pd

import datetime, calendar

import matplotlib.pyplot as plt
import mplcursors

import plotly.graph_objects as go

from PIL import Image
from io import BytesIO
import base64

import ipywidgets as widgets
from IPython.display import display, Markdown, HTML

import RAYearlyStats_backend as RA

################################
### HTML auxiliary functions ###
################################

def HTML_code_show_picture(picture):

    buffer = BytesIO()
    picture.savefig(buffer, format="png", bbox_inches="tight")
    buffer.seek(0)
    picture_base64 = base64.b64encode(buffer.read()).decode("utf-8")
    buffer.close()

    return f"""<img src="data:image/png;base64,{picture_base64}" style="width: 100%; height: auto; display: block;" alt="Figure">"""

def HTML_draw_horizontal_line():

    display(HTML("""<div style="width: 100%; height: 1px; background-color: black;"></div>"""))

#################
### Main code ###
#################

df_historic = RA.retrieve_historic_df(username=username,
                                      api_key=api_key,
                                      hardcore_mode_only=hardcore_mode_only,
                                      )

df_awards = RA.retrieve_awards_df(username=username,
                                  api_key=api_key,
                                  )

df_events = RA.get_event_data(df_historic, drop=True)

df_games_data, cheevos_data_dict = RA.retrieve_necessary_games_data(df_historic=df_historic,
                                                                    api_key=api_key,
                                                                    )

year_list = list(df_historic["Year"].unique())

default_year = datetime.datetime.now().year - 1
if default_year not in year_list:
    default_year = year_list[-1]

@widgets.interact(year=widgets.Dropdown(options=year_list, value=default_year, description='Year:', disabled=False))
def show_yearly_stats(year):

    ######################
    ### Initialization ###
    ######################

    # Retrieve/calculate necessary data

    user_icon = RA.get_user_icon_fig(username)

    stats     = RA.get_yearly_stats(df_historic, df_awards, year)
    dev_stats = RA.get_yearly_favdev_stats(df_historic, year)

    # Parameters

    title_fontsize = 36
    section_title_fontsize = 28
    general_text_fontsize = 18

    # Template for four column displays

    base_html_code = ["""<div style="display: flex; justify-content: space-between; align-items: center; width: 100%; font-family: Arial, sans-serif;">""",
                      """<!-- Column 1 -->""",
                      """<div style="width: 8%; text-align: center;">""",
                      "",
                      """</div>""",
                      """<!-- Column 2 -->""",
                      """<div style="width: 42%; text-align: left;">""",
                      "",
                      """</div>""",
                      """<!-- Column 3 -->""",
                      """<div style="width: 8%; text-align: center;">""",
                      "",
                      """</div>""",
                      """<!-- Column 4 -->""",
                      """<div style="width: 42%; text-align: left;">""",
                      "",
                      """</div>""",
                      """</div>"""]

    # Proper spacing with widget
    
    print()

    #############
    ### Title ###
    #############

    display(HTML(f"""
        <p style="font-size: {title_fontsize}px; margin-bottom: 10px;">
            <b>{username}'s {year} in RetroAchievements</b>
        </p>
    """))

    HTML_draw_horizontal_line()

    ####################
    ### General data ###
    ####################
    
    display(HTML(f"""
        <p style="font-size: 18px; margin-bottom: 10px;">
            Achievements earned: <b>{stats["Achievements total"]}</b>
        </p>
    """))

    display(HTML(f"""
        <p style="font-size: 18px; margin-bottom: 10px;">
            Points earned: <b>{stats["Points total"]} ({stats["RetroPoints total"]})</b>
        </p>
    """))

    if not hardcore_mode_only:

        display(HTML(f"""
            <p style="font-size: 18px; margin-bottom: 10px;">
                Softcore Points earned: <b>{stats["Softcore Points total"]}</b>
            </p>
        """))

    display(HTML(f"""
        <p style="font-size: 18px; margin-bottom: 10px;">
            Games with at least one achievement earned: <b>{stats["Game total"]}</b>
        </p>
    """))

    display(HTML(f"""
        <p style="font-size: 18px; margin-bottom: 10px;">
            Beaten games: <b>{len(stats["Beaten games"])}</b>
        </p>
    """))

    display(HTML(f"""
        <p style="font-size: 18px; margin-bottom: 10px;">
            Mastered games: <b>{len(stats["Mastered games"])}</b>
        </p>
    """))

    HTML_draw_horizontal_line()
    
    ####################
    ### Beaten games ###
    ####################

    if len(stats["Beaten games"]) > 0:

        display(HTML(f"""
            <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;">
                <b>Beaten games</b>
            </p>
        """))

        for i in range(0,len(stats["Beaten games"]),2):

            # First column

            title = stats["Beaten games"].loc[i, "Title"]
            console = stats["Beaten games"].loc[i,"ConsoleName"]
            date = RA.get_formatted_date(stats["Beaten games"].loc[i,"Day"], stats["Beaten games"].loc[i,"Month"])
            game_icon = stats["Game icons"][stats["Beaten games"].loc[i, "AwardData"]]
    
            base_html_code[3] = HTML_code_show_picture(game_icon)
            base_html_code[7] = f"""
                <p style="font-size: 16px; margin-bottom: 10px;">{title}</p>
                <p style="font-size: 12px;">{date} | {console}</p>
            """
    
            # Second column
    
            if i+1 < len(stats["Beaten games"]):
    
                title = stats["Beaten games"].loc[i+1, "Title"]
                console = stats["Beaten games"].loc[i+1,"ConsoleName"]
                date = RA.get_formatted_date(stats["Beaten games"].loc[i+1,"Day"], stats["Beaten games"].loc[i+1,"Month"])
                game_icon = stats["Game icons"][stats["Beaten games"].loc[i+1, "AwardData"]]
    
                base_html_code[11] = HTML_code_show_picture(game_icon)
                base_html_code[15] = f"""
                    <p style="font-size: 16px; margin-bottom: 10px;">{title}</p>
                    <p style="font-size: 12px;">{date} | {console}</p>
                """
    
            else:
    
                base_html_code[11] = """&nbsp;"""
                base_html_code[15] = """&nbsp;"""
    
            display(HTML("".join(base_html_code)))
    
        HTML_draw_horizontal_line()

    ######################
    ### Mastered games ###
    ######################

    if len(stats["Mastered games"]) > 0:

        display(HTML(f"""
            <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;">
                <b>Mastered games</b>
            </p>
        """))

        for i in range(0,len(stats["Mastered games"]),2):

            # First column
    
            title = stats["Mastered games"].loc[i, "Title"]
            console = stats["Mastered games"].loc[i,"ConsoleName"]
            date = RA.get_formatted_date(stats["Mastered games"].loc[i,"Day"], stats["Mastered games"].loc[i,"Month"])
            game_icon = stats["Game icons"][stats["Mastered games"].loc[i, "AwardData"]]
    
            base_html_code[3] = HTML_code_show_picture(game_icon)
            base_html_code[7] = f"""
                <p style="font-size: 16px; margin-bottom: 10px;">{title}</p>
                <p style="font-size: 12px;">{date} | {console}</p>
            """
    
            # Second column
    
            if i+1 < len(stats["Mastered games"]):
    
                title = stats["Mastered games"].loc[i+1, "Title"]
                console = stats["Mastered games"].loc[i+1,"ConsoleName"]
                date = RA.get_formatted_date(stats["Mastered games"].loc[i+1,"Day"], stats["Mastered games"].loc[i+1,"Month"])
                game_icon = stats["Game icons"][stats["Mastered games"].loc[i+1, "AwardData"]]
    
                base_html_code[11] = HTML_code_show_picture(game_icon)
                base_html_code[15] = f"""
                    <p style="font-size: 16px; margin-bottom: 10px;">{title}</p>
                    <p style="font-size: 12px;">{date} | {console}</p>
                """
    
            else:
    
                base_html_code[11] = """&nbsp;"""
                base_html_code[15] = """&nbsp;"""
    
            display(HTML("".join(base_html_code)))
    
        HTML_draw_horizontal_line()

    ################################
    ### Daily point distribution ###
    ################################

    display(HTML(f"""
        <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;">
            <b>Daily point distribution</b>
        </p>
    """))

    fig = RA.get_figure_daily_points_one_year(df_historic, year)
    fig.show()

    HTML_draw_horizontal_line()

    ###################################################
    ### Achievements with highest RetroPoint value  ###
    ###################################################

    display(HTML(f"""
        <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;">
            <b>Achievements with highest RetroPoint value</b>
        </p>
    """))

    for i in range(0,len(stats["Hardest achievements"]),2):

        # First column

        achievement = stats["Hardest achievements"][i]
        badge_icon  = stats["Hardest achievements badges"][i]

        base_html_code[3] = HTML_code_show_picture(badge_icon)
        base_html_code[7] = f"""
            <p style="font-size: 16px; margin-bottom: 10px;">{achievement["Title"]} | {achievement["Points"]} ({achievement["TrueRatio"]})</p>
            <p style="font-size: 12px;">{RA.get_game_title(achievement["GameID"], df_games_data)}</p>
        """

        # Second column

        if i+1 < len(stats["Hardest achievements"]):

            achievement = stats["Hardest achievements"][i+1]
            badge_icon = stats["Hardest achievements badges"][i+1]

            base_html_code[11] = HTML_code_show_picture(badge_icon)
            base_html_code[15] = f"""
                <p style="font-size: 16px; margin-bottom: 10px;">{achievement["Title"]} | {achievement["Points"]} ({achievement["TrueRatio"]})</p>
                <p style="font-size: 12px;">{RA.get_game_title(achievement["GameID"], df_games_data)}</p>
            """

        else:

            base_html_code[11] = """&nbsp;"""
            base_html_code[15] = """&nbsp;"""

        display(HTML("".join(base_html_code)))

    HTML_draw_horizontal_line()

    ###########################################
    ### Achievement distribution by console ###
    ###########################################

    display(HTML(f"""
        <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;">
            <b>Achievement distribution by console</b>
        </p>
    """))

    fig = RA.get_figure_system_distribution(df_historic, year, by="Achievements")
    fig.show()

    HTML_draw_horizontal_line()

    #############################################
    ### Achievement distribution by developer ###
    #############################################

    display(HTML(f"""
        <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;">
            <b>Achievement distribution by developer</b>
        </p>
    """))

    fig = RA.get_figure_dev_distribution(df_historic, year, by="Achievements")
    fig.show()

    HTML_draw_horizontal_line()

    ##########################
    ### Favorite developer ###
    ##########################

    dev_username = dev_stats["Username"]
    dev_cheevo_total = dev_stats["Achievement total"]
    dev_percent = dev_stats["Achievement %"]

    display(HTML(f"""
        <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;">
            <b>Favorite developer appreciation corner</b>
        </p>
    """))

    display(HTML(f"""
        <div style="display: flex; justify-content: space-between; align-items: center; width: 100%; font-family: Arial, sans-serif;">
            <!-- Column 1 -->
            <div style="width: 15%; text-align: left;">
                {HTML_code_show_picture(dev_stats["User icon"])}
            </div>
    
            <!-- Column 2 -->
            <div style="width: 85%; text-align: left;">
                <p style="font-size: {section_title_fontsize}px; margin-bottom: 10px;"><b>{dev_username}</b></p>
            </div>
        </div>
    """))

    display(HTML(f"""
        <p style="font-size: {general_text_fontsize}px; margin-bottom: 10px;">
            {dev_username} developed <b>{dev_cheevo_total} achievements</b> of the {stats["Achievements total"]} that {username} got in {year}. That's the <b>{dev_percent:.2f} %</b>!
        </p>
        <p style="font-size: {general_text_fontsize}px; margin-bottom: 10px;">
           They sure deserve a thank you for making {year} more enjoyable!
        </p>
        <p style="font-size: {general_text_fontsize}px; margin-bottom: 10px;">
            Here's a breakdown of the achievements developed by them that {username} played in {year}:
        </p>
    """))

    for i in range(0,len(dev_stats["Game distribution"]),2):

        # First column

        game_id = dev_stats["Game distribution"].index[i]
        
        if game_id in stats["Game icons"].keys():
            game_icon = stats["Game icons"][game_id]
        else:
            game_icon = RA.get_game_icon_fig(df_historic, game_id)

        base_html_code[3] = HTML_code_show_picture(game_icon)
        base_html_code[7] = f"""
            <p style="font-size: 16px; margin-bottom: 10px;">{RA.get_game_title(game_id, df_games_data)}</p>
            <p style="font-size: 12px;">{RA.get_game_console(game_id, df_historic)}, {dev_stats["Game distribution"].iloc[i]} achievements</p>
        """

        # Second column

        if i+1 < len(dev_stats["Game distribution"]):

            game_id = dev_stats["Game distribution"].index[i+1]
        
            if game_id in stats["Game icons"].keys():
                game_icon = stats["Game icons"][game_id]
            else:
                game_icon = RA.get_game_icon_fig(df_historic, game_id)

            base_html_code[11] = HTML_code_show_picture(game_icon)
            base_html_code[15] = f"""
                <p style="font-size: 16px; margin-bottom: 10px;">{RA.get_game_title(game_id, df_games_data)}</p>
                <p style="font-size: 12px;">{RA.get_game_console(game_id, df_historic)}, {dev_stats["Game distribution"].iloc[i+1]} achievements</p>
            """

        else:

            base_html_code[11] = """&nbsp;"""
            base_html_code[15] = """&nbsp;"""

        display(HTML("".join(base_html_code)))